Gaze Estimation RIGHT & LEFT FEATURE

In [1]:
# Import necessary libraries
import cv2
import mediapipe as mp
import numpy as np
import math

In [3]:
############## PARAMETERS ############
# facial landmarks
#Left Eye Coordinates
LEFT_EYE = [ 362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385,384, 398 ]
#Right Eye Coordinates
RIGHT_EYE=[ 33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161 , 246 ]
#IRISH POSITIONS
RIGHT_IRISH= [474, 475, 476, 477]
LEFT_IRISH = [469, 470, 471, 472]
#RIGHT EYE EDGE POSITIONS
RIGHT_EYE_RIGHTMOST = [33]  # right eye right most landmark 
RIGHT_EYE_LEFTMOST = [133]  # right eye left most landmark
RIGHT_EYE_TOPMOST = [159]   # right eye TOP most landmark
RIGHT_EYE_BOTTOMMOST = [145] # right eye BOTTOM most landmark

#LEFT EYE EDGE POSITIONS
LEFT_EYE_RIGHTMOST = [362]  # left eye right most landmark
LEFT_EYE_LEFTMOST = [263]  # left eye left most landmark
LEFT_EYE_TOPMOST = [386]   # left eye TOP most landmark
LEFT_EYE_BOTTOMMOST = [374] # left eye BOTTOM most landmark

#################################################################################

In [4]:
#Euclidean_Distance_Measurement_between2 Points
def eucledian_distance( point1 , point2):
    x1, y1 = point1.ravel()
    x2, y2 = point2.ravel()
    #Calculating_The_Distance
    distance = math.sqrt(pow(x2-x1,2)+pow(y2-y1,2))
    return distance

#IRISH_POSITIONS
def irish_position(irish_center, irish_right_point, irish_left_point):
    center_to_Right_distance = eucledian_distance (irish_center, irish_right_point)
    total_distance = eucledian_distance(irish_right_point,irish_left_point)
    ratio = (center_to_Right_distance/total_distance)
    irish_position = ""
    if ratio<= 0.42:
        irish_position= "RIGHT"
    elif ratio > 0.42 and ratio<= 0.53:     #DECIDING ACCORDING TO ERROR
        irish_position = "CENTER"
    else:
        irish_position = "LEFT"
    
    return irish_position, ratio*100

In [8]:
# Initialize MediaPipe FaceMesh and OpenCV VideoCapture
mp_face_mesh = mp.solutions.face_mesh
cap = cv2.VideoCapture(0)

# Configure FaceMesh with specified parameters
with mp_face_mesh.FaceMesh(
    static_image_mode=False,
    refine_landmarks=True,
    max_num_faces=2,
    min_detection_confidence=0.5,
    min_tracking_confidence= 0.5
) as face_mesh:
    while True:
        ret,frame =cap.read()   # Read a frame from the camera
        if not ret:
            break
        # Flip the frame and convert from BGR to RGB
        frame = cv2.flip(frame, 1)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Get the height and width of the frame. [:2] for only h,w leaving RGB val
        img_h, img_w =frame.shape[:2]

        # Process the frame with FaceMesh
        results = face_mesh.process(frame)

        # Check if there are multiple face landmarks detected
        if results.multi_face_landmarks:
            # Extract pixel coordinates of facial landmarks
            mesh_points = np.array(
                [
                    np.multiply([p.x,p.y], [img_w,img_h]).astype(int)
                    for p in results.multi_face_landmarks[0].landmark
                ],
            # print(mesh_points) # Print for debugging purposes
            )

            # Draw circles, calculate Irish position, and display text
            (l_cx, l_cy), l_radius = cv2.minEnclosingCircle(mesh_points[LEFT_IRISH]) #l_cx = left center x,y cordinate
            (r_cx, r_cy), r_radius = cv2.minEnclosingCircle(mesh_points[RIGHT_IRISH]) #r_cx = right center x,y cordinate
            #IRISH Center Position
            center_right = np.array([l_cx, l_cy], dtype=np.int32) #as we have mirrored the image changing the variables
            
            center_left = np.array([r_cx, r_cy], dtype=np.int32) #as we have mirrored the image changing the variables
            #print(center_left)

            #DRAW EYELINES
            cv2.polylines(frame, [mesh_points[LEFT_EYE]],True,(0,255,0),1,cv2.LINE_AA)
            cv2.polylines(frame, [mesh_points[RIGHT_EYE]],True,(0,255,0),1,cv2.LINE_AA)

            # Draw circles on the frame (IRISH)
            cv2.circle(frame, center_left, int(l_radius), (3, 252, 227), 1, cv2.LINE_AA)
            cv2.circle(frame, center_right, int(r_radius), (3, 252, 227), 1, cv2.LINE_AA)

            #Draw pointers on eye LEFT
            cv2.circle(
                frame, mesh_points[LEFT_EYE_LEFTMOST][0], 1, (245, 2, 2), -1, cv2.LINE_AA
            )
            print(mesh_points[LEFT_EYE_LEFTMOST])
            cv2.circle(
                frame, mesh_points[LEFT_EYE_RIGHTMOST][0], 1, (245, 2, 2), 1, cv2.LINE_AA
            )
            cv2.circle(
                frame, mesh_points[LEFT_EYE_TOPMOST][0], 1, (245, 2, 2), 1, cv2.LINE_AA
            )
            cv2.circle(
                frame, mesh_points[LEFT_EYE_BOTTOMMOST][0], 1, (245, 2, 2), 1, cv2.LINE_AA
            )
            #Draw pointers on eye RIGHT
            cv2.circle(
                frame, mesh_points[RIGHT_EYE_RIGHTMOST][0], 1, (245, 2, 2), -1, cv2.LINE_AA
            )
            cv2.circle(
                frame, mesh_points[RIGHT_EYE_LEFTMOST][0], 1, (245, 2, 2), 1, cv2.LINE_AA
            )
            cv2.circle(
                frame, mesh_points[RIGHT_EYE_TOPMOST][0], 1, (245, 2, 2), 1, cv2.LINE_AA
            )
            cv2.circle(
                frame, mesh_points[RIGHT_EYE_BOTTOMMOST][0], 1, (245, 2, 2), 1, cv2.LINE_AA
            )
            
            # GET Irish position
            irish_pos , ratio = irish_position(

                center_left, mesh_points[LEFT_EYE_LEFTMOST], mesh_points[LEFT_EYE_RIGHTMOST][0]    #As_Per_mirroring
            )
            #print(mesh_points[LEFT_EYE_RIGHTMOST])
            # Display Irish position text on the frame
            cv2.putText(
                frame,
                f"IRISH POSITION:{irish_pos} {ratio:.2f}%",
                (30,450),
                cv2.FONT_HERSHEY_COMPLEX,
                1,
                (245, 149, 66),
                2,
                cv2.LINE_AA,
            )
        # Display the frame with annotations
        cv2.imshow('Gaze Detection', frame)
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break
# Release the camera and close all windows
cap.release()
cv2.destroyAllWindows()

[[277 178]]
[[275 178]]
[[275 179]]
[[274 179]]
[[274 179]]
[[274 179]]
[[274 180]]
[[274 180]]
[[274 180]]
[[274 180]]
[[274 181]]
[[274 181]]
[[274 181]]
[[274 181]]
[[274 181]]
[[274 181]]
[[274 181]]
[[274 181]]
[[274 181]]
[[274 181]]
[[274 181]]
[[275 181]]
[[275 181]]
[[276 180]]
[[279 180]]
[[278 180]]
[[282 182]]
[[282 181]]
[[282 180]]
[[281 180]]
[[281 179]]
[[280 180]]
[[281 180]]
[[281 180]]
[[281 179]]
[[282 179]]
[[282 179]]
[[282 179]]


USING_DIFFERENT_APPROACH_TO_ADD_UP&DOWN_FEATURES

The FUNCTION -"IRISH POSITION" has been updated

In [9]:
# Euclidean Distance Measurement between 2 Points
def eucledian_distance(point1, point2):
    x1, y1 = point1.ravel()
    x2, y2 = point2.ravel()
    distance = math.sqrt(pow(x2 - x1, 2) + pow(y2 - y1, 2))
    return distance

# Updated IRISH_POSITIONS function
def irish_position_UPDATED(center, left, right, topmost, bottommost):
    distances = {
        'left': eucledian_distance(center, left),
        'right': eucledian_distance(center, right),
        'top': eucledian_distance(center, topmost),
        'bottom': eucledian_distance(center, bottommost),
    }

    # Find the key with the minimum value in the distances dictionary
    min_distance_key = min(distances, key=distances.get)

    return min_distance_key, distances[min_distance_key]

IMPLEMENTING THE PREVIOUS PORTION OF CODE

In [10]:
# Initialize MediaPipe FaceMesh and OpenCV VideoCapture
mp_face_mesh = mp.solutions.face_mesh
cap = cv2.VideoCapture(0)

# Configure FaceMesh with specified parameters
with mp_face_mesh.FaceMesh(
    static_image_mode=False,
    refine_landmarks=True,
    max_num_faces=2,
    min_detection_confidence=0.5,
    min_tracking_confidence= 0.5
) as face_mesh:
    while True:
        ret,frame =cap.read()   # Read a frame from the camera
        if not ret:
            break
        # Flip the frame and convert from BGR to RGB
        frame = cv2.flip(frame, 1)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Get the height and width of the frame. [:2] for only h,w leaving RGB val
        img_h, img_w =frame.shape[:2]

        # Process the frame with FaceMesh
        results = face_mesh.process(frame)

        # Check if there are multiple face landmarks detected
        if results.multi_face_landmarks:
            # Extract pixel coordinates of facial landmarks
            mesh_points = np.array(
                [
                    np.multiply([p.x,p.y], [img_w,img_h]).astype(int)
                    for p in results.multi_face_landmarks[0].landmark
                ],
            # print(mesh_points) # Print for debugging purposes
            )

            # Draw circles, calculate Irish position, and display text
            (l_cx, l_cy), l_radius = cv2.minEnclosingCircle(mesh_points[LEFT_IRISH]) #l_cx = left center x,y cordinate
            (r_cx, r_cy), r_radius = cv2.minEnclosingCircle(mesh_points[RIGHT_IRISH]) #r_cx = right center x,y cordinate
            #IRISH Center Position
            center_right = np.array([l_cx, l_cy], dtype=np.int32) #as we have mirrored the image changing the variables
            
            center_left = np.array([r_cx, r_cy], dtype=np.int32) #as we have mirrored the image changing the variables
            #print(center_left)

            #DRAW EYELINES
            cv2.polylines(frame, [mesh_points[LEFT_EYE]],True,(0,255,0),1,cv2.LINE_AA)
            cv2.polylines(frame, [mesh_points[RIGHT_EYE]],True,(0,255,0),1,cv2.LINE_AA)

            # Draw circles on the frame (IRISH)
            cv2.circle(frame, center_left, int(l_radius), (3, 252, 227), 1, cv2.LINE_AA)
            cv2.circle(frame, center_right, int(r_radius), (3, 252, 227), 1, cv2.LINE_AA)

            #Draw pointers on eye LEFT
            cv2.circle(
                frame, mesh_points[LEFT_EYE_LEFTMOST][0], 1, (245, 2, 2), -1, cv2.LINE_AA
            )
            print(mesh_points[LEFT_EYE_LEFTMOST])
            cv2.circle(
                frame, mesh_points[LEFT_EYE_RIGHTMOST][0], 1, (245, 2, 2), 1, cv2.LINE_AA
            )
            cv2.circle(
                frame, mesh_points[LEFT_EYE_TOPMOST][0], 1, (245, 2, 2), 1, cv2.LINE_AA
            )
            cv2.circle(
                frame, mesh_points[LEFT_EYE_BOTTOMMOST][0], 1, (245, 2, 2), 1, cv2.LINE_AA
            )
            #Draw pointers on eye RIGHT
            cv2.circle(
                frame, mesh_points[RIGHT_EYE_RIGHTMOST][0], 1, (245, 2, 2), -1, cv2.LINE_AA
            )
            cv2.circle(
                frame, mesh_points[RIGHT_EYE_LEFTMOST][0], 1, (245, 2, 2), 1, cv2.LINE_AA
            )
            cv2.circle(
                frame, mesh_points[RIGHT_EYE_TOPMOST][0], 1, (245, 2, 2), 1, cv2.LINE_AA
            )
            cv2.circle(
                frame, mesh_points[RIGHT_EYE_BOTTOMMOST][0], 1, (245, 2, 2), 1, cv2.LINE_AA
            )
            # GET Irish position
            irish_pos, distance = irish_position_UPDATED(
                center_left,
                mesh_points[LEFT_EYE_LEFTMOST][0],
                mesh_points[LEFT_EYE_RIGHTMOST][0],
                mesh_points[LEFT_EYE_TOPMOST][0],
                mesh_points[LEFT_EYE_BOTTOMMOST][0]
            )

            # Display Irish position text on the frame
            cv2.putText(
                frame,
                f"IRISH POSITION: {irish_pos} {distance:.2f}",
                (30, 450),
                cv2.FONT_HERSHEY_COMPLEX,
                1,
                (245, 149, 66),
                2,
                cv2.LINE_AA,
            )

        # Display the frame with annotations
        cv2.imshow('Gaze Detection', frame)
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

# Release the camera and close all windows
cap.release()
cv2.destroyAllWindows()

[[270 185]]
[[271 185]]
[[271 185]]
[[271 186]]
[[271 185]]
[[271 186]]
[[271 186]]
[[272 186]]
[[272 186]]
[[272 186]]
[[271 186]]
[[271 186]]
[[275 188]]
[[273 186]]
[[273 186]]
[[273 185]]
[[273 185]]
[[273 183]]
[[273 184]]
[[275 182]]
[[274 182]]
[[276 179]]
[[276 179]]
[[278 177]]
[[280 176]]
[[280 176]]
[[282 176]]
[[281 177]]
[[283 176]]
[[283 176]]
[[284 173]]
[[284 173]]
[[285 173]]
[[285 173]]
[[286 172]]
[[286 172]]
[[288 171]]
[[290 168]]
[[291 168]]
[[293 165]]
[[294 165]]
[[297 159]]
[[297 160]]
[[300 156]]
[[302 153]]
[[302 152]]
[[305 152]]
[[305 152]]
[[308 152]]
[[308 152]]
[[310 153]]
[[311 155]]
[[312 155]]
[[311 157]]
[[312 157]]
[[310 159]]
[[310 159]]
[[309 161]]
[[309 161]]
[[309 163]]
[[308 163]]
[[308 163]]
[[308 164]]
[[308 164]]
[[309 165]]
[[309 165]]
[[310 165]]
[[311 165]]
[[310 166]]
[[311 166]]
[[311 166]]
[[312 166]]
[[312 165]]
[[312 164]]
[[312 164]]
[[312 163]]
[[313 163]]
[[312 162]]
[[313 162]]
[[313 161]]
[[313 161]]
[[314 159]]
[[313 157]]
[[31